In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb


def f(data):
    text = [i['text'] for i in data]
    label = [i['label'] for i in data]

    data = tokenizer(text, device=device)
    data['labels'] = torch.FloatTensor(label).to(device)

    return data


loader = get_loader(f, negative_label=True, with_answer=True)

len(loader), next(iter(loader))

(62500,
 {'input_ids': tensor([[ 0, 13, 12,  4,  6, 14, 10,  8,  6, 11, 18,  5, 10,  6, 12, 13,  1,  2,
            2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
            2,  2,  2],
          [ 0,  6, 13, 12, 11, 14, 10,  8,  9,  4, 18, 13,  8,  7, 11, 14, 13,  7,
            7, 18,  5,  4,  7, 11,  4, 14, 13,  7,  5,  8, 18,  5, 13, 11,  4, 13,
            1,  2,  2],
          [ 0,  8,  4,  4, 11, 14,  5, 13, 10,  5, 18,  9, 13, 10, 12, 14,  8, 12,
           13,  8, 18,  5,  4, 13, 10,  9,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
            2,  2,  2],
          [ 0,  7,  8, 13, 10, 14,  9,  4,  4,  4, 18, 12,  8, 13, 10, 14,  7,  8,
           13, 12, 18,  5,  6,  5,  6,  6,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
            2,  2,  2],
          [ 0,  5,  5,  7, 11, 14,  8,  9,  4, 18,  5,  9, 12, 11, 14, 10,  8,  7,
           12, 18, 12,  4,  6,  9,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
            2,  2,  2],
          [ 0,  9,  4, 12,  9

In [3]:
%run 3.model.ipynb

model_critic = Gemma3Critic(len(tokenizer), tokenizer.pad_token_id,tokenizer.eos_token_id).to(device)
#model_critic = torch.load('model/critic', weights_only=False).to(device)

sum(i.numel() for i in model_critic.parameters()) / 1_0000_0000

2.01422848

In [4]:
optimizer = torch.optim.Adam(model_critic.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                              base_lr=1e-5,
                                              max_lr=1e-4,
                                              step_size_up=1000)

for epoch in range(40_0000):
    data = next(iter(loader))

    loss, logits = model_critic(**data)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    if epoch % 1000 == 0:
        logits = (logits > 0.5).squeeze(1).long()
        acc = (logits == data['labels'].long()).sum() / len(data['labels'])
        print(epoch, loss.item(), acc.item())

torch.save(model_critic.to('cpu'), 'model/critic')

0 0.4070695638656616 0.375
1000 0.38209378719329834 0.25
2000 0.25080031156539917 0.4375
3000 0.24669183790683746 0.5625
4000 0.2549736201763153 0.375
5000 0.24508115649223328 0.5625
6000 0.25359800457954407 0.25
7000 0.24961596727371216 0.625
8000 0.24944931268692017 0.5625
9000 0.26551973819732666 0.4375
10000 0.2512258291244507 0.4375
11000 0.30359578132629395 0.5
12000 0.2586905062198639 0.375
13000 0.2434312105178833 0.625
14000 0.2505418062210083 0.5
15000 0.2539946734905243 0.5
16000 0.2564588785171509 0.4375
17000 0.2804393172264099 0.4375
18000 0.249833345413208 0.5
19000 0.24675226211547852 0.5625
20000 0.2516259253025055 0.375
21000 0.2735432982444763 0.3125
22000 0.25803452730178833 0.375
23000 0.2546844482421875 0.375
24000 0.248661071062088 0.5625
25000 0.24559956789016724 0.5625
26000 0.24690701067447662 0.6875
27000 0.24715930223464966 0.625
28000 0.24775150418281555 0.625
29000 0.2503160536289215 0.5625
30000 0.2540351152420044 0.375
31000 0.2560177147388458 0.375
3200

253000 0.0532238744199276 0.9375
254000 0.061535656452178955 0.9375
255000 0.24492290616035461 0.6875
256000 0.0774487555027008 0.875
257000 0.06755175441503525 0.9375
258000 0.06499262899160385 0.875
259000 0.12868434190750122 0.8125
260000 0.00470309890806675 1.0
261000 0.08985453099012375 0.875
262000 0.1072244793176651 0.875
263000 0.052289895713329315 0.9375
264000 0.07268092781305313 0.9375
265000 0.006905793212354183 1.0
266000 0.007543587125837803 1.0
267000 0.05773168429732323 0.9375
268000 0.08362363278865814 0.875
269000 0.1362980306148529 0.8125
270000 0.02100067026913166 1.0
271000 0.06300139427185059 0.9375
272000 0.019401244819164276 0.9375
273000 0.03244801238179207 0.9375
274000 0.05958474427461624 0.9375
275000 0.09385383129119873 0.875
276000 0.13079625368118286 0.8125
277000 0.0077892797999084 1.0
278000 0.003704980481415987 1.0
279000 0.057150889188051224 0.9375
280000 0.005467685405164957 1.0
281000 0.1163824275135994 0.8125
282000 0.0032481704838573933 1.0
283000